In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
import json
from pathlib import Path

# # define groq key
# groq_key = (
#      os.environ['groqkey']
# )
# groq_key

In [2]:
from dotenv import load_dotenv

load_dotenv()

groq_key = os.environ['groqkey']

data_path = Path(os.environ['data_path'])


In [3]:
from extraction import llmAgent
from query import DataLoader
from main import main

/Users/reedevans/Documents/mids/deprescribe_main/DataSci210_MedicationDeprescriber/src/extraction.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
recommendation_dict = {
        "continue": [
            "Barretts Esophagus or esophageal cell changes",
            "Chronic Non-Steroidal Anti Inflammatory (NSAID) use or GI prophylaxis NSAID use",
            "Severe esophagitis including bleeding esophagitis or esophageal ulcer",
            "History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage",
        ],
        "stop": [
            "Peptic Ulcer Disease or Gastroduodenal ulcer treated for 2 - 12 weeks caused from H Pylori infection or NSAID use without bleeding",
            "Upper GI Symptoms such as reflux, difficulty swallowing, nausea, or vomiting without endoscopy - asymptomatic for 3 consecutive days",
            "ICU Stress Ulcer Prophylaxis",
            "Completed Heliobacter Pylori (H. Pylori) infection treated for 14 days with combination therapy",
        ],
        "deprescribe": [
            "Mild to moderate esophagitis or esophageal inflammation",
            "Treated Gastroesophageal Reflux Disease (GERD) or reflux symptoms such as acid reflux, heartburn, or regurgitation",
        ],
    }

In [5]:
encounter_key = "D6253A5CE371EA"

In [6]:
final_recommendation, final_reasoning, token_usage, search_history_so_far, token_count_history = main(
    groq_key=groq_key, data_path=data_path, encounter_key=encounter_key
)

In [7]:
final_recommendation

'continue'

In [8]:
final_reasoning 

"The patient's PPI medication should be continued due to a documented history of gastrointestinal bleeding (GIB) on admit, which supports its ongoing use."

In [22]:
from fuzzywuzzy import process  # Import fuzzy matching library

diagnosis_label_key = {
    0:"Barretts Esophagus or esophageal cell changes",
    1:"Chronic Non-Steroidal Anti Inflammatory (NSAID) use or GI prophylaxis NSAID use",
    2:"Severe esophagitis including bleeding esophagitis or esophageal ulcer",
    3:"History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage",
    4:"Peptic Ulcer Disease or Gastroduodenal ulcer treated for 2 - 12 weeks caused from H Pylori infection or NSAID use without bleeding",
    5:"Upper GI Symptoms such as reflux, difficulty swallowing, nausea, or vomiting without endoscopy - asymptomatic for 3 consecutive days",
    6:"ICU Stress Ulcer Prophylaxis",
    7:"Completed Heliobacter Pylori (H. Pylori) infection treated for 14 days with combination therapy",
    8:"Mild to moderate esophagitis or esophageal inflammation",
    9:"Treated Gastroesophageal Reflux Disease (GERD) or reflux symptoms such as acid reflux, heartburn, or regurgitation",
    10:"No clear evidence found"
}

def get_best_match(reasoning, diagnosis_dict):
    # Extract the diagnosis labels from the dictionary
    labels = list(diagnosis_dict.values())
    # Use fuzzy matching to find the best match
    from difflib import get_close_matches
    best_match = get_close_matches(reasoning, labels, n=1, cutoff=0.6) 
    matched_key = list(diagnosis_dict.keys())[list(diagnosis_dict.values()).index(best_match[0])] if best_match else None
    
    
    return matched_key, best_match[0] # Return the matched key and label

# Call the function with final_reasoning and diagnosis_label_key
matched_diagnosis, best_match = get_best_match(final_reasoning, diagnosis_label_key)
print(matched_diagnosis, ":",best_match)

IndexError: list index out of range

In [10]:
get_best_match('History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage',diagnosis_label_key)

'History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage'

In [ ]:
diagnosis_label_key = {
    0:"Barretts Esophagus or esophageal cell changes",
    2:"Severe esophagitis including bleeding esophagitis or esophageal ulcer",
    3:"History of gastrointestinal bleeding, gastric ulcer, upper GI bleed, or peptic ulcer hemorrhage",
    4:"Peptic Ulcer Disease or Gastroduodenal ulcer treated for 2 - 12 weeks caused from H Pylori infection or NSAID use without bleeding",
    5:"Upper GI Symptoms such as reflux, difficulty swallowing, nausea, or vomiting without endoscopy - asymptomatic for 3 consecutive days",
    6:"ICU Stress Ulcer Prophylaxis",
    7:"Completed Heliobacter Pylori (H. Pylori) infection treated for 14 days with combination therapy",
    8:"Mild to moderate esophagitis or esophageal inflammation",
    9:"Treated Gastroesophageal Reflux Disease (GERD) or reflux symptoms such as acid reflux, heartburn, or regurgitation",
    10:"No clear evidence found"
}